In [ ]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd
from faker import Faker
fake = Faker()

In [ ]:
engine = create_engine(connstr)

In [ ]:
conn = engine.connect()
query = """ \
SELECT * FROM vwQuestions
WHERE PK_FORM LIKE '%2022-A-IG%'
"""  
df = pd.read_sql(query,con=conn) 
conn.close()
df_vwQuestions=df

In [ ]:
conn = engine.connect()
query = """ \
SELECT * FROM  vwOrgSubToComponent 
WHERE PK_FORM LIKE '%2022-A-IG%'
AND Acronym = 'SBA'
"""  
df = pd.read_sql(query,con=conn) 
conn.close()
df_vwOrgSubToComponent=df

In [ ]:
df

In [ ]:
for i,r in df_vwQuestions.iterrows():
    s = ''
    s = s + f" -- {str(r['PK'])}  {str(r['QGROUP'])}"
    print( s ) 
 

In [ ]:
def getip(): 
    return f"{np.random.randint(1, 255)}.{np.random.randint(1, 255)}.{np.random.randint(1, 255)}.{np.random.randint(1, 255)}"
getip()